In [ ]:
import requests
import pandas as pd
import datetime
import json
import sqlite3

# DB_FILE = "/Users/lulutsai/Documents/NTPU_class/paper/code/mobile01_clawler/ntpu_paper.sqlite"
DB_FILE = "D:/NTPU_class/paper/code/mobile01_clawler/ntpu_paper.sqlite"
API_TOKEN = 'sk-a94e722134454665b8fef72ddb349f37'
API_URL = "http://localhost:3000/api/v1/chat/completions"
LIMIT_COUNT = 2 # ← 你想一次處理幾筆資料（可自行調整）
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_TOKEN}"
}


# === 🔹 Function 1: 確保欄位存在 ===
def ensure_column(conn, table, column, definition):
    cur = conn.cursor()
    cur.execute(f"PRAGMA table_info({table});")
    columns = [c[1] for c in cur.fetchall()]
    if column not in columns:
        print(f"🧱 新增 {table}.{column} 欄位...")
        cur.execute(f"ALTER TABLE {table} ADD COLUMN {column} {definition};")
        conn.commit()
        print(f"✅ 已新增欄位 {column}")
    else:
        print(f"🔎 欄位 {column} 已存在，略過新增")


# === 🔹 Function 2: 建立分析表（若不存在） ===
def ensure_articles_pua_table(conn):
    cur = conn.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS articles_pua (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        link_id TEXT NOT NULL,
        article_id TEXT NOT NULL,
        level TEXT DEFAULT 'article',  -- 預留給未來 comment/thread 共用
        score_overall INTEGER,
        score_cold INTEGER,
        score_threat INTEGER,
        score_blame INTEGER,
        score_guilt INTEGER,
        score_devalue INTEGER,
        score_control INTEGER,
        score_victim INTEGER,
        score_other INTEGER,
        main_type TEXT,
        reason TEXT,
        tone_summary TEXT,
        confidence REAL,
        model_name TEXT,
        model_version TEXT,
        knowledge_base TEXT,
        created_at TEXT,
        FOREIGN KEY (article_id) REFERENCES articles(id),
        FOREIGN KEY (link_id) REFERENCES links(id)
    );
    """)
    conn.commit()
    print("✅ 確認資料表 articles_pua 已存在")


# === 🔹 Function 3: 取得待分析的主文 ===
def get_pending_articles(conn, limit):
    query = f"""
    SELECT id, link_id, content
    FROM articles
    WHERE content IS NOT NULL 
      AND TRIM(content) != ''
      AND (pua_status IS NULL OR pua_status = 'pending')
    LIMIT {limit};
    """
    df = pd.read_sql_query(query, conn)
    print(f"📘 共讀取 {len(df)} 筆文章待分析（上限 {limit}）")
    return df


# === 🔹 Function 4: 更新分析狀態 ===
def update_article_status(conn, article_id, status):
    cur = conn.cursor()
    cur.execute("UPDATE articles SET pua_status = ? WHERE id = ?;", (status, article_id))
    conn.commit()


# === 🔹 Function 5: 呼叫模型進行情緒勒索分析 ===
def analyze_text(text):
    data = {
        "model": "mistral:latest",
        "knowledge": ["pua_db"],
        "messages": [
            {
                "role": "system",
                "content": """你是一個以繁體中文回覆的情緒勒索分析助手。
                請根據知識庫文件，對給定文字做以下任務，並輸出 JSON 格式：
                {
                    "score_overall": 整體分數(1~10),
                    "score_cold": 冷暴力分數,
                    "score_threat": 威脅分數,
                    "score_blame": 責怪分數,
                    "score_guilt": 罪惡感操控分數,
                    "score_devalue": 貶低分數,
                    "score_control": 控制分數,
                    "score_victim": 扮可憐分數,
                    "score_other": 其他類型分數,
                    "main_type": "主要勒索類型（以繁體中文表示）",
                    "reason": "一句話說明為什麼這樣判斷（繁體中文）",
                    "tone_summary": "整體語氣摘要（可省略）",
                    "confidence": 模型對此次判斷的信心水準(0~1之間)
                }
                ⚠️ 請務必只輸出純 JSON，內容請使用繁體中文。
                不要輸出任何英文字說明或非 JSON 格式文字。
                """
            },
            {"role": "user", "content": f"請分析這段文字: {text}"}
        ]
    }

    res = requests.post(API_URL, headers=headers, json=data, timeout=90)
    content = res.json()["choices"][0]["message"]["content"]
    return json.loads(content)



# === 🔹 Function 6: 寫入分析結果 ===
def insert_analysis_result(conn, link_id, article_id, result):
    cur = conn.cursor()
    created_at = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    cur.execute("""
        INSERT INTO articles_pua (
            link_id, article_id, level,
            score_overall, score_cold, score_threat, score_blame, score_guilt,
            score_devalue, score_control, score_victim, score_other,
            main_type, reason, tone_summary, confidence,
            model_name, model_version, knowledge_base, created_at
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
    """, (
        link_id, article_id, result.get("level", "article"),
        result.get("score_overall"),
        result.get("score_cold"),
        result.get("score_threat"),
        result.get("score_blame"),
        result.get("score_guilt"),
        result.get("score_devalue"),
        result.get("score_control"),
        result.get("score_victim"),
        result.get("score_other"),
        result.get("main_type"),
        result.get("reason"),
        result.get("tone_summary"),
        result.get("confidence"),
        result.get("model_name", "mistral"),
        result.get("model_version", "latest"),
        result.get("knowledge_base", "pua_db"),
        created_at
    ))

    conn.commit()


# === 🔸 主程式 ===
def main():
    conn = sqlite3.connect(DB_FILE)

    # 建立欄位與分析表
    ensure_column(conn, "articles", "pua_status", "TEXT DEFAULT 'pending'")
    ensure_articles_pua_table(conn)

    # 抓待分析文章
    df = get_pending_articles(conn, LIMIT_COUNT)
    if df.empty:
        print("⚠️ 沒有待分析的資料。")
        conn.close()
        return

    for idx, row in df.iterrows():
        link_id = row["link_id"]
        article_id = row["id"]
        text = row["content"]

        try:
            result = analyze_text(text)
            insert_analysis_result(conn, link_id, article_id, result)
            update_article_status(conn, article_id, "done")
            print(f"✅ 第 {idx+1}/{len(df)} 筆完成 | overall={result.get('score_overall')} | type={result.get('main_type')}")

        except Exception as e:
            update_article_status(conn, article_id, "error")
            print(f"❌ 第 {idx+1}/{len(df)} 筆失敗：{e}")

    conn.close()
    print("🎉 全部完成！")



# === 執行 ===
if __name__ == "__main__":
    main()

🔎 欄位 pua_status 已存在，略過新增
✅ 確認資料表 articles_pua 已存在
📘 共讀取 2 筆文章待分析（上限 2）
 {
    "score_overall": 8,
    "score_cold": 3,
    "score_threat": 7,
    "score_blame": 8,
    "score_guilt": 5,
    "score_devalue": 6,
    "score_control": 9,
    "score_victim": 4,
    "score_other": 2,
    "main_type": "情緒勒索",
    "reason": "文字描述了家人使用情感黑暗、控制和責怪來影響主人公，並展示了主人公對於這種勒索感到無力。",
    "tone_summary": "悲哀和畏懼",
    "confidence": 0.95
}
✅ 第 1/2 筆完成 | overall=8 | type=情緒勒索
 {
    "score_overall": 7,
    "score_cold": 8,
    "score_threat": 6,
    "score_blame": 9,
    "score_guilt": 4,
    "score_devalue": 5,
    "score_control": 10,
    "score_victim": 3,
    "score_other": 2,
    "main_type": "Manipulation and Control",
    "reason": "The text describes a relationship where one person (the woman) consistently exerts control over the other (the man), causing distress and self-blame.",
    "tone_summary": "Dominating, accusatory, self-pitying",
    "confidence": 0.85
}
✅ 第 2/2 筆完成 | overall=7 | type=Manipul